In [2]:
import cv2
import os
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [3]:
mp_face_mesh = mp.solutions.face_mesh
mp_face_detect = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

### code for hand landmarks

In [ ]:
capture = cv2.VideoCapture(0)
with mp_hands.Hands(model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while capture.isOpened():
        success, image = capture.read()
        if not success:
            print("Empty camera frame")
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image) # for face detection

        # drawing face detection annotations on the image
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        # code for showing hand landmarks
        if results.multi_hand_landmarks:
            for hand in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                              mp_drawing_styles.get_default_hand_landmarks_style(),
                                            mp_drawing_styles.get_default_hand_connections_style())
        

        cv2.imshow("Hand landmarks", cv2.flip(image, 1))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    capture.release()
    cv2.destroyAllWindows()

# code for holistic

In [4]:
def landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                            mp_drawing.DrawingSpec(color=(60,255,122),thickness=1,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing_styles.get_default_hand_landmarks_style(),
                                            mp_drawing_styles.get_default_hand_connections_style())
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing_styles.get_default_hand_landmarks_style(),
                                            mp_drawing_styles.get_default_hand_connections_style())

In [5]:
def detect_holistic(image, mp_model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = mp_model.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [9]:
capture = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while capture.isOpened():
        success, image = capture.read()
        if not success:
            print("Empty camera frame")
            continue
            
        image, results = detect_holistic(image, holistic) # for all detection

        landmarks(image,results)
        
        cv2.imshow("holistic landmarks", cv2.flip(image, 1)) 

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    capture.release()
    cv2.destroyAllWindows()

In [34]:
results.face_landmarks.landmark

[x: 0.5624995827674866
y: 0.6418834924697876
z: -0.024279916658997536
, x: 0.5596131682395935
y: 0.6007000803947449
z: -0.043742354959249496
, x: 0.5604234933853149
y: 0.6136462092399597
z: -0.023565983399748802
, x: 0.5503764748573303
y: 0.5653476119041443
z: -0.031943269073963165
, x: 0.5588845610618591
y: 0.5890694856643677
z: -0.04625207558274269
, x: 0.5579337477684021
y: 0.5745900869369507
z: -0.04261209815740585
, x: 0.5557940006256104
y: 0.5409243702888489
z: -0.019981030374765396
, x: 0.4981450140476227
y: 0.5438268184661865
z: 0.012727166526019573
, x: 0.5543918609619141
y: 0.517822265625
z: -0.013919379562139511
, x: 0.5536459684371948
y: 0.5036114454269409
z: -0.015179140493273735
, x: 0.5505428314208984
y: 0.4474947154521942
z: -0.00380929303355515
, x: 0.5628955364227295
y: 0.6475774645805359
z: -0.023248329758644104
, x: 0.5631186366081238
y: 0.6519051194190979
z: -0.02059016562998295
, x: 0.5632206797599792
y: 0.653712272644043
z: -0.017014266923069954
, x: 0.5637736320

In [14]:
pos = np.array([[each.x, each.y, each.z, each.visibility] for each in results.pose_landmarks.landmark]).flatten()

In [16]:
len(pos)

132

In [26]:
lh = np.array([[each.x, each.y, each.z] for each in results.left_hand_landmarks.landmark ]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

In [28]:
rh = np.array([[each.x, each.y, each.z] for each in results.right_hand_landmarks.landmark ]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [29]:
rh

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [33]:
pos = np.array([[each.x, each.y, each.z, each.visibility] for each in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)

#### Extract holistic keypoints and stack them together

In [6]:
def holistic_keypoints(results):
    pos = np.array([[each.x, each.y, each.z, each.visibility] for each in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[each.x, each.y, each.z] for each in results.left_hand_landmarks.landmark ]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[each.x, each.y, each.z] for each in results.right_hand_landmarks.landmark ]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    face = np.array([[each.x, each.y, each.z] for each in results.face_landmarks.landmark ]).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([pos,face,lh,rh])

In [7]:
path_data = os.path.join('Video data Mediapipe')

In [7]:
signs = np.array(['Start', 'End', 'Hello', 'Dad', 'Mom', 'Hungry', 'Thank you', 'I Love You'])

In [8]:
no_videos = 30  #30 videos worth of data
no_frames = 30 #each video with 30 frames

In [45]:
#Folder structure

#Sign 1
#->0
#->1
#->2
#...
#Sign 2
#->0
#->1
#->2
#...

In [18]:
for sign in signs:
    for frame in range(no_frames):
        try:
            os.makedirs(os.path.join(path_data, sign, str(frame)))
        except:
            pass

In [29]:
capture = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while capture.isOpened():
        for sign in signs:
            for video in range(no_videos):
                for frame in range(no_frames):
                    
                    success, image = capture.read()

                    image, results = detect_holistic(image, holistic) # for all detection

                    landmarks(image,results)
                    
                    if frame == 0:
                        cv2.putText(image, f'STARTING COLLECTION for {sign}', (120,120), cv2.FONT_HERSHEY_PLAIN, 1, (0,250,0), 5, cv2.LINE_AA)
                        cv2.putText(image, f'Collecting frames for {sign} video number {video}', (20,15), cv2.FONT_HERSHEY_PLAIN, 0.4, (255,0,0), 2, cv2.LINE_AA)
                        cv2.waitKey(3000)
                        
                    else:
                        cv2.putText(image, f'Collecting frames for {sign} video number {video}', (20,15), cv2.FONT_HERSHEY_PLAIN, 0.4, (255,0,0), 2, cv2.LINE_AA)
                        
                    cv2.imshow('collecting images', image)
                    
                    stacked_result = holistic_keypoints(results)
                    numpy_path = os.path.join(path_data, sign, str(video), str(frame))
                    print("Data getting saved in ",numpy_path)
                    np.save(numpy_path, stacked_result)

#                     cv2.imshow("holistic landmarks", cv2.flip(image, 1))
                    
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
    capture.release()
    cv2.destroyAllWindows()

Data getting saved in  Video data Mediapipe\Start\0\0
Data getting saved in  Video data Mediapipe\Start\0\1
Data getting saved in  Video data Mediapipe\Start\0\2
Data getting saved in  Video data Mediapipe\Start\0\3
Data getting saved in  Video data Mediapipe\Start\0\4
Data getting saved in  Video data Mediapipe\Start\0\5
Data getting saved in  Video data Mediapipe\Start\0\6
Data getting saved in  Video data Mediapipe\Start\0\7
Data getting saved in  Video data Mediapipe\Start\0\8
Data getting saved in  Video data Mediapipe\Start\0\9
Data getting saved in  Video data Mediapipe\Start\0\10
Data getting saved in  Video data Mediapipe\Start\0\11
Data getting saved in  Video data Mediapipe\Start\0\12
Data getting saved in  Video data Mediapipe\Start\0\13
Data getting saved in  Video data Mediapipe\Start\0\14
Data getting saved in  Video data Mediapipe\Start\0\15
Data getting saved in  Video data Mediapipe\Start\0\16
Data getting saved in  Video data Mediapipe\Start\0\17
Data getting saved i

Data getting saved in  Video data Mediapipe\Start\5\0
Data getting saved in  Video data Mediapipe\Start\5\1
Data getting saved in  Video data Mediapipe\Start\5\2
Data getting saved in  Video data Mediapipe\Start\5\3
Data getting saved in  Video data Mediapipe\Start\5\4
Data getting saved in  Video data Mediapipe\Start\5\5
Data getting saved in  Video data Mediapipe\Start\5\6
Data getting saved in  Video data Mediapipe\Start\5\7
Data getting saved in  Video data Mediapipe\Start\5\8
Data getting saved in  Video data Mediapipe\Start\5\9
Data getting saved in  Video data Mediapipe\Start\5\10
Data getting saved in  Video data Mediapipe\Start\5\11
Data getting saved in  Video data Mediapipe\Start\5\12
Data getting saved in  Video data Mediapipe\Start\5\13
Data getting saved in  Video data Mediapipe\Start\5\14
Data getting saved in  Video data Mediapipe\Start\5\15
Data getting saved in  Video data Mediapipe\Start\5\16
Data getting saved in  Video data Mediapipe\Start\5\17
Data getting saved i

Data getting saved in  Video data Mediapipe\Start\10\0
Data getting saved in  Video data Mediapipe\Start\10\1
Data getting saved in  Video data Mediapipe\Start\10\2
Data getting saved in  Video data Mediapipe\Start\10\3
Data getting saved in  Video data Mediapipe\Start\10\4
Data getting saved in  Video data Mediapipe\Start\10\5
Data getting saved in  Video data Mediapipe\Start\10\6
Data getting saved in  Video data Mediapipe\Start\10\7
Data getting saved in  Video data Mediapipe\Start\10\8
Data getting saved in  Video data Mediapipe\Start\10\9
Data getting saved in  Video data Mediapipe\Start\10\10
Data getting saved in  Video data Mediapipe\Start\10\11
Data getting saved in  Video data Mediapipe\Start\10\12
Data getting saved in  Video data Mediapipe\Start\10\13
Data getting saved in  Video data Mediapipe\Start\10\14
Data getting saved in  Video data Mediapipe\Start\10\15
Data getting saved in  Video data Mediapipe\Start\10\16
Data getting saved in  Video data Mediapipe\Start\10\17
Da

Data getting saved in  Video data Mediapipe\Start\14\29
Data getting saved in  Video data Mediapipe\Start\15\0
Data getting saved in  Video data Mediapipe\Start\15\1
Data getting saved in  Video data Mediapipe\Start\15\2
Data getting saved in  Video data Mediapipe\Start\15\3
Data getting saved in  Video data Mediapipe\Start\15\4
Data getting saved in  Video data Mediapipe\Start\15\5
Data getting saved in  Video data Mediapipe\Start\15\6
Data getting saved in  Video data Mediapipe\Start\15\7
Data getting saved in  Video data Mediapipe\Start\15\8
Data getting saved in  Video data Mediapipe\Start\15\9
Data getting saved in  Video data Mediapipe\Start\15\10
Data getting saved in  Video data Mediapipe\Start\15\11
Data getting saved in  Video data Mediapipe\Start\15\12
Data getting saved in  Video data Mediapipe\Start\15\13
Data getting saved in  Video data Mediapipe\Start\15\14
Data getting saved in  Video data Mediapipe\Start\15\15
Data getting saved in  Video data Mediapipe\Start\15\16
Da

Data getting saved in  Video data Mediapipe\Start\19\28
Data getting saved in  Video data Mediapipe\Start\19\29
Data getting saved in  Video data Mediapipe\Start\20\0
Data getting saved in  Video data Mediapipe\Start\20\1
Data getting saved in  Video data Mediapipe\Start\20\2
Data getting saved in  Video data Mediapipe\Start\20\3
Data getting saved in  Video data Mediapipe\Start\20\4
Data getting saved in  Video data Mediapipe\Start\20\5
Data getting saved in  Video data Mediapipe\Start\20\6
Data getting saved in  Video data Mediapipe\Start\20\7
Data getting saved in  Video data Mediapipe\Start\20\8
Data getting saved in  Video data Mediapipe\Start\20\9
Data getting saved in  Video data Mediapipe\Start\20\10
Data getting saved in  Video data Mediapipe\Start\20\11
Data getting saved in  Video data Mediapipe\Start\20\12
Data getting saved in  Video data Mediapipe\Start\20\13
Data getting saved in  Video data Mediapipe\Start\20\14
Data getting saved in  Video data Mediapipe\Start\20\15
Da

...

Data getting saved in  Video data Mediapipe\I Love You\13\29
Data getting saved in  Video data Mediapipe\I Love You\14\0
Data getting saved in  Video data Mediapipe\I Love You\14\1
Data getting saved in  Video data Mediapipe\I Love You\14\2
Data getting saved in  Video data Mediapipe\I Love You\14\3
Data getting saved in  Video data Mediapipe\I Love You\14\4
Data getting saved in  Video data Mediapipe\I Love You\14\5
Data getting saved in  Video data Mediapipe\I Love You\14\6
Data getting saved in  Video data Mediapipe\I Love You\14\7
Data getting saved in  Video data Mediapipe\I Love You\14\8
Data getting saved in  Video data Mediapipe\I Love You\14\9
Data getting saved in  Video data Mediapipe\I Love You\14\10
Data getting saved in  Video data Mediapipe\I Love You\14\11
Data getting saved in  Video data Mediapipe\I Love You\14\12
Data getting saved in  Video data Mediapipe\I Love You\14\13
Data getting saved in  Video data Mediapipe\I Love You\14\14
Data getting saved in  Video data 

Data getting saved in  Video data Mediapipe\I Love You\18\16
Data getting saved in  Video data Mediapipe\I Love You\18\17
Data getting saved in  Video data Mediapipe\I Love You\18\18
Data getting saved in  Video data Mediapipe\I Love You\18\19
Data getting saved in  Video data Mediapipe\I Love You\18\20
Data getting saved in  Video data Mediapipe\I Love You\18\21
Data getting saved in  Video data Mediapipe\I Love You\18\22
Data getting saved in  Video data Mediapipe\I Love You\18\23
Data getting saved in  Video data Mediapipe\I Love You\18\24
Data getting saved in  Video data Mediapipe\I Love You\18\25
Data getting saved in  Video data Mediapipe\I Love You\18\26
Data getting saved in  Video data Mediapipe\I Love You\18\27
Data getting saved in  Video data Mediapipe\I Love You\18\28
Data getting saved in  Video data Mediapipe\I Love You\18\29
Data getting saved in  Video data Mediapipe\I Love You\19\0
Data getting saved in  Video data Mediapipe\I Love You\19\1
Data getting saved in  Vid

Data getting saved in  Video data Mediapipe\I Love You\23\5
Data getting saved in  Video data Mediapipe\I Love You\23\6
Data getting saved in  Video data Mediapipe\I Love You\23\7
Data getting saved in  Video data Mediapipe\I Love You\23\8
Data getting saved in  Video data Mediapipe\I Love You\23\9
Data getting saved in  Video data Mediapipe\I Love You\23\10
Data getting saved in  Video data Mediapipe\I Love You\23\11
Data getting saved in  Video data Mediapipe\I Love You\23\12
Data getting saved in  Video data Mediapipe\I Love You\23\13
Data getting saved in  Video data Mediapipe\I Love You\23\14
Data getting saved in  Video data Mediapipe\I Love You\23\15
Data getting saved in  Video data Mediapipe\I Love You\23\16
Data getting saved in  Video data Mediapipe\I Love You\23\17
Data getting saved in  Video data Mediapipe\I Love You\23\18
Data getting saved in  Video data Mediapipe\I Love You\23\19
Data getting saved in  Video data Mediapipe\I Love You\23\20
Data getting saved in  Video 

Data getting saved in  Video data Mediapipe\I Love You\27\21
Data getting saved in  Video data Mediapipe\I Love You\27\22
Data getting saved in  Video data Mediapipe\I Love You\27\23
Data getting saved in  Video data Mediapipe\I Love You\27\24
Data getting saved in  Video data Mediapipe\I Love You\27\25
Data getting saved in  Video data Mediapipe\I Love You\27\26
Data getting saved in  Video data Mediapipe\I Love You\27\27
Data getting saved in  Video data Mediapipe\I Love You\27\28
Data getting saved in  Video data Mediapipe\I Love You\27\29
Data getting saved in  Video data Mediapipe\I Love You\28\0
Data getting saved in  Video data Mediapipe\I Love You\28\1
Data getting saved in  Video data Mediapipe\I Love You\28\2
Data getting saved in  Video data Mediapipe\I Love You\28\3
Data getting saved in  Video data Mediapipe\I Love You\28\4
Data getting saved in  Video data Mediapipe\I Love You\28\5
Data getting saved in  Video data Mediapipe\I Love You\28\6
Data getting saved in  Video da

KeyboardInterrupt: 

In [9]:
label_map = {label:num for num, label in enumerate(signs)}

In [10]:
label_map

{'Start': 0,
 'End': 1,
 'Hello': 2,
 'Dad': 3,
 'Mom': 4,
 'Hungry': 5,
 'Thank you': 6,
 'I Love You': 7}

In [35]:
videos, labels = [], []
for sign in signs:
    for video in range(no_videos):
        total_video = []
        for frame in range(no_frames):
            res = np.load(os.path.join(path_data, sign, str(video), "{}.npy".format(frame)))
            total_video.append(res)
        videos.append(total_video)
        labels.append(label_map[sign])

In [39]:
# 8 signs : 240 videos , 30 frames each, each frame 1662 array size
np.array(videos).shape

(240, 30, 1662)

In [40]:
np.array(labels).shape

(240,)

In [42]:
X = np.array(videos)
y = to_categorical(labels).astype(int)

In [132]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# LSTM Neural network

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model

In [134]:
log_dir = os.path.join('Logs')
callback = TensorBoard(log_dir=log_dir)

In [138]:
model = Sequential()
model.add(LSTM(64, return_sequences = True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences = True, activation='relu'))
model.add(LSTM(64, return_sequences = False, activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(signs.shape[0], activation='softmax'))

In [139]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [140]:
model.fit(X_train,y_train, epochs=360, callbacks=[callback])

Epoch 1/360
8/8 [==============================] - 3s 156ms/step - loss: 4.2121 - categorical_accuracy: 0.1009
Epoch 2/360
8/8 [==============================] - 1s 132ms/step - loss: 2.2640 - categorical_accuracy: 0.1053
Epoch 3/360
8/8 [==============================] - 1s 136ms/step - loss: 2.0598 - categorical_accuracy: 0.1316
Epoch 4/360
8/8 [==============================] - 1s 163ms/step - loss: 2.0809 - categorical_accuracy: 0.2237
Epoch 5/360
8/8 [==============================] - 1s 124ms/step - loss: 2.2361 - categorical_accuracy: 0.1711
Epoch 6/360
8/8 [==============================] - 1s 115ms/step - loss: 1.7161 - categorical_accuracy: 0.3377
Epoch 7/360
8/8 [==============================] - 1s 125ms/step - loss: 1.8031 - categorical_accuracy: 0.2763
Epoch 8/360
8/8 [==============================] - 1s 119ms/step - loss: 2.0601 - categorical_accuracy: 0.1579
Epoch 9/360
8/8 [==============================] - 1s 134ms/step - loss: 2.0536 - categorical_accuracy: 0.1535
E

8/8 [==============================] - 1s 126ms/step - loss: 0.5260 - categorical_accuracy: 0.7939
Epoch 147/360
8/8 [==============================] - 1s 131ms/step - loss: 0.5136 - categorical_accuracy: 0.7895
Epoch 148/360
8/8 [==============================] - 1s 128ms/step - loss: 0.5085 - categorical_accuracy: 0.8246
Epoch 149/360
8/8 [==============================] - 1s 121ms/step - loss: 0.4849 - categorical_accuracy: 0.8114
Epoch 150/360
8/8 [==============================] - 1s 123ms/step - loss: 0.5867 - categorical_accuracy: 0.7105
Epoch 151/360
8/8 [==============================] - 1s 120ms/step - loss: 0.5692 - categorical_accuracy: 0.7456
Epoch 152/360
8/8 [==============================] - 1s 125ms/step - loss: 0.5134 - categorical_accuracy: 0.7807
Epoch 153/360
8/8 [==============================] - 1s 118ms/step - loss: 0.9111 - categorical_accuracy: 0.7105
Epoch 154/360
8/8 [==============================] - 1s 125ms/step - loss: 0.6611 - categorical_accuracy: 0.70

8/8 [==============================] - 1s 125ms/step - loss: 0.4453 - categorical_accuracy: 0.8158
Epoch 291/360
8/8 [==============================] - 1s 130ms/step - loss: 0.4151 - categorical_accuracy: 0.8158
Epoch 292/360
8/8 [==============================] - 1s 144ms/step - loss: 0.4201 - categorical_accuracy: 0.8202
Epoch 293/360
8/8 [==============================] - 1s 129ms/step - loss: 0.3787 - categorical_accuracy: 0.8333
Epoch 294/360
8/8 [==============================] - 1s 115ms/step - loss: 0.4629 - categorical_accuracy: 0.7851
Epoch 295/360
8/8 [==============================] - 1s 121ms/step - loss: 0.4746 - categorical_accuracy: 0.7632
Epoch 296/360
8/8 [==============================] - 1s 123ms/step - loss: 0.4358 - categorical_accuracy: 0.7939
Epoch 297/360
8/8 [==============================] - 1s 130ms/step - loss: 0.3875 - categorical_accuracy: 0.8333
Epoch 298/360
8/8 [==============================] - 1s 139ms/step - loss: 0.4767 - categorical_accuracy: 0.79

In [163]:
model.optimizer.lr

<tf.Variable 'Adam/learning_rate:0' shape=() dtype=float32, numpy=0.001>

In [66]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_17 (LSTM)              (None, 30, 64)            442112    
                                                                 
 lstm_18 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_19 (LSTM)              (None, 64)                49408     
                                                                 
 dense_16 (Dense)            (None, 64)                4160      
                                                                 
 dense_17 (Dense)            (None, 32)                2080      
                                                                 
 dense_18 (Dense)            (None, 8)                 264       
                                                                 
Total params: 596,840
Trainable params: 596,840
Non-tr

In [141]:
res = model.predict(X_test)

In [160]:
list(zip(signs[np.argmax(res,axis=1)],signs[np.argmax(y_test,axis=1)]))

[('Dad', 'Dad'),
 ('Hungry', 'Hungry'),
 ('Dad', 'Dad'),
 ('I Love You', 'I Love You'),
 ('Hungry', 'Start'),
 ('Mom', 'Mom'),
 ('Start', 'Start'),
 ('Thank you', 'Thank you'),
 ('End', 'Hello'),
 ('End', 'Start'),
 ('Mom', 'Mom'),
 ('Start', 'End')]

In [144]:
yhat = np.argmax(res,axis=1).tolist()

In [145]:
ytrue = np.argmax(y_test,axis=1).tolist()

In [146]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [148]:
accuracy_score(yhat,ytrue)

0.6666666666666666

In [149]:
model.save('signs.h5')

In [13]:
model = load_model('signs.h5')

In [14]:
import time

In [24]:
video = []
signs = []

capture = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while capture.isOpened():
        for frame in no_frames:
            if frame==0:
                cv2.putText(image, f'STARTING COLLECTION for {sign}', (120,120), cv2.FONT_HERSHEY_PLAIN, 1, (0,250,0), 5, cv2.LINE_AA)
                
            success, image = capture.read()
        
            
            image, results = detect_holistic(image, holistic) # for all detection
        
            landmarks(image,results)
        
            keypoints = holistic_keypoints(results)
            video.append(keypoints)
#         video = video[-30:] # taking continous last 30 frames
        
        if len(video) == 30:
            sign = model.predict(np.expand_dims(video, axis=0))[0]
            print([k for k,v in label_map.items() if v == np.argmax(sign)])
            frame = 0
        
        
        cv2.imshow("holistic landmarks", image) 
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    capture.release()
    cv2.destroyAllWindows()

['Mom']
['Mom']
['Mom']
['Mom']
['Mom']
['Mom']
['Mom']
['Start']
['Start']
['Start']
['Start']
['Start']
['Hungry']
['Hungry']
['Mom']
['Mom']
['Start']
['Start']
['Start']
['Start']
['Mom']
['Start']
['Hungry']
['Start']
['Hungry']
['Mom']
['Hungry']
['Mom']
['Hungry']
['Mom']
['Mom']
['Mom']
['Mom']
['Mom']
['Start']
['Start']
['Start']
['Start']
['Start']


KeyboardInterrupt: 